# Primera Parte
## Limpieza y procesamiento de datos

<img src="./logo_banguat_final.png">

Guatemala recibió en febrero de 2023 US$ 1,381.1 millones por remesasfamiliares, que equivale
a unos Q 10mil 910 millones. Las remesas son una fuente importante en la economía
guatemalteca, en el enlace: http://www.banguat.gob.gt/es/page/anos-2002-2023 podrá
encontrar un cuadro con el histórico de remesas familiares en Guatemala, cuadro que deberá
de normalizar respecto a la fecha para luego realizar las siguientes operaciones en columnas
aparte, tomar en cuenta que esta base de datos será actualizada recurrentemente.
- Crecimiento trimestral
- Crecimiento interanual
- Media móvil de 6 meses
Además de realizar una gráfica por cada columna (Valor, crecimientos, media móvil)
Detallar cada paso realizado en la operación, queries y formulas utilizadas.

### Librerias utilizadas para la limpieza de datos y obtencion de tabla principal

In [134]:
import openpyxl
import requests
import pandas as pd

### Consideracion de actualización de base de datos recurrentemente

Cómo la base de datos se va actualizando, no es viable la descarga directa y trabajar sobre el .xlsx sin actualizaciones. Necesitamos una forma de obtener siempre la última versión de este archivo y esto se logra solicitando el HTTP request directamente a la pagina y almacenando el archivo .xslx que nos envien localmente. Este proceso se realizará cada vez que se ejecute este Jupyter Notebook

In [135]:
# URL con el cual realizamos el HTTP request
url = "http://www.banguat.gob.gt/sites/default/files/banguat/estaeco/remesas/remfam2010_2023.xlsx"

# Obtencion del nombre
filename = url.split('/')[-1]
response = requests.get(url)

# Escritura del archivo localmente
with open(filename, 'wb') as f:
    f.write(response.content)

# Obtencion de df especifica del excel y de la worksheet
wb = openpyxl.load_workbook("remfam2010_2023.xlsx")
ws = wb['2002-2023']

# Valores en los que se encuentra la tabla que nos interesa
df = ws['B10':'X22']
rows = []
for row in df:
    rows.append([cell.value for cell in row])
df = pd.DataFrame(rows)

# Escribir el dfFrame en un archivo CSV
df.to_csv('remfam2010_2023_clean.csv', index=False, header=False)

# # Cargar el archivo CSV
df = pd.read_csv('remfam2010_2023_clean.csv', index_col=0)

# Realizar la suma en pandas ya que el valor devuelto del .xslx no es el valor sino la formula
total_row = df.sum(axis=0)
total_row.name = "Total"
df = pd.concat([df, total_row.to_frame().T])

# Guardar el archivo actualizado
df.to_csv('remfam2010_2023_clean.csv')
print("Archivo actualizado exitosamente.")

Archivo actualizado exitosamente.


### Leemos el archivo csv limpio

En este se encuentra nuestra tabla "Cuadro 1 Guatemala: Ingreso de Divisas por Remesas Familiares En millones de US dólares	Años: 2002 - 2023" que obtuvimos del .xlsx actualizable. Con esta información ya podemos empezar a realizar las operaciones que nos solicitan

In [136]:
df = pd.read_csv('remfam2010_2023_clean.csv', index_col=0)
print(df[['2002.0','2003.0','2004.0']])

                2002.0      2003.0      2004.0
Enero         83.15574   164.75655   194.74402
Febrero       96.65939   144.74341   183.73886
Marzo        109.03374   160.77811   227.89545
Abril        139.21199   181.08733   204.01319
Mayo          89.75174   187.37634   210.78039
Junio        127.97632   161.49457   212.12977
Julio        136.52537   191.95433   206.61293
Agosto       162.30790   178.91320   212.78174
Septiembre   157.33877   176.55097   211.77725
Octubre      176.26081   189.88092   218.02659
Noviembre    142.97583   173.37563   236.70772
Diciembre    158.19428   195.59340   231.41521
Total       1579.39188  2106.50476  2550.62312


### Normalización


In [137]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

# Cargar los datos del archivo CSV en un dfFrame de Pandas
df = pd.read_csv('remfam2010_2023_clean.csv', index_col=0)


"""Normalizacion respecto al valor final del año"""
df = df.div(df.iloc[-1], axis=1)
print(df[['2002.0','2003.0','2004.0']])

df.to_csv('remfam2010_2023_normalizado.csv')


"""Normalizacion con sklearn"""
# scaler = MinMaxScaler()
# df_norm = pd.DataFrame(scaler.fit_transform(df), columns=df.columns, index=df.index)

# df_norm.to_csv('remfam2010_2023_normalizado.csv')

# print(df_norm[['2002.0','2003.0','2004.0']])

              2002.0    2003.0    2004.0
Enero       0.052650  0.078213  0.076352
Febrero     0.061200  0.068713  0.072037
Marzo       0.069035  0.076325  0.089349
Abril       0.088143  0.085966  0.079986
Mayo        0.056827  0.088951  0.082639
Junio       0.081029  0.076665  0.083168
Julio       0.086442  0.091125  0.081005
Agosto      0.102766  0.084934  0.083423
Septiembre  0.099620  0.083812  0.083030
Octubre     0.111600  0.090140  0.085480
Noviembre   0.090526  0.082305  0.092804
Diciembre   0.100162  0.092852  0.090729
Total       1.000000  1.000000  1.000000


'Normalizacion con sklearn'

### Datos solicitados
Estos se encuentran dentro del archivo 'remfam2010_2023_datos2.csv' el cual luego de realizar las tazas solicitadas y agregarlas dentro del csv permite realizar las graficas. Para que esta implementación tome en cuenta datos actuales recopilados del .xslx mas reciente es necesario exportar los datos del Excel 'remfam2010_2023_datos.xlsx' a 'remfam2010_2023_datos2.xlsx' que es en donde se realizan las operaciones

In [138]:
# Convertir a archivo excel que nosotros modificaremos
df = pd.read_csv('remfam2010_2023_normalizado.csv')
df.to_excel('remfam2010_2023_datos.xlsx', index=False)


Recreación de operaciones automaticamente para no estar realizando cambios entre excels y procesos de copy paste que pueden afectar el proceso de alguna manera

In [139]:

wb = openpyxl.load_workbook('remfam2010_2023_datos.xlsx')
ws = wb['Sheet1']
ws['A15'] = 'Q1' 
ws['A16'] = 'Q2' 
ws['A17'] = 'Q3' 
ws['A18'] = 'Q4'
ws['A19'] = 'TazaCrecimiento'
ws['A20'] = 'TCAQ1' 
ws['A21'] = 'TCAQ2' 
ws['A22'] = 'TCAQ3' 
ws['A23'] = 'TCAQ4'
ws['A24'] = 'MM_ENE'
ws['A25'] = 'MM_FEB'
ws['A26'] = 'MM_MAR'
ws['A27'] = 'MM_ABRIL'
ws['A28'] = 'MM_MAY'
ws['A29'] = 'MM_JUN'
ws['A30'] = 'MM_JUL'
ws['A31'] = 'MM_AG'
ws['A32'] = 'MM_SEP'
ws['A33'] = 'MM_OCT'
ws['A34'] = 'MM_NOV'
ws['A35'] = 'MM_DIC'

ws['B15'] = '=SUM(B2:B4)'
ws['B16'] = "=((SUM(B5:B7)-SUM(B2:B4))/SUM(B2:B4))"
ws['B17'] = "=((SUM(B8:B10)-SUM(B5:B7))/SUM(B5:B7))"
ws['B18'] = "=((SUM(B11:B13)-SUM(B8:B10))/SUM(B8:B10))"
ws['B20'] = 0
ws['B21'] = 0
ws['B22'] = 0
ws['B23'] = 0
ws['B24'] = '=AVERAGE(B2)'
ws['B25'] = '=AVERAGE(B2:B3)'
ws['B26'] = '=AVERAGE(B2:B4)'
ws['B27'] = '=AVERAGE(B2:B5)'
ws['B28'] = '=AVERAGE(B2:B6)'
ws['B29'] = '=AVERAGE(B2:B7)'
ws['B30'] = '=AVERAGE(B3:B8)'
ws['B31'] = '=AVERAGE(B4:B9)'
ws['B32'] = '=AVERAGE(B5:B10)'
ws['B33'] = '=AVERAGE(B6:B11)'
ws['B34'] = '=AVERAGE(B7:B12)'
ws['B35'] = '=AVERAGE(B8:B13)'


for col in range(3,23):
    for row_i in range(15,19):
        if row_i == 15:
            ws.cell(row=row_i, column=col).value = f'=(SUM({ws.cell(row=row_i, column=(col)).column_letter}2:{ws.cell(row=row_i, column=(col)).column_letter}4)-SUM({ws.cell(row=row_i, column=(col-1)).column_letter}11:{ws.cell(row=row_i, column=(col-1)).column_letter}13))/SUM({ws.cell(row=row_i, column=(col-1)).column_letter}11:{ws.cell(row=row_i, column=(col-1)).column_letter}13)'
        if row_i == 16:
            ws.cell(row=row_i, column=col).value = f'=((SUM({ws.cell(row=row_i, column=(col)).column_letter}5:{ws.cell(row=row_i, column=(col)).column_letter}7)-SUM({ws.cell(row=row_i, column=(col)).column_letter}2:{ws.cell(row=row_i, column=(col)).column_letter}4))/SUM({ws.cell(row=row_i, column=(col)).column_letter}2:{ws.cell(row=row_i, column=(col)).column_letter}4))'
        if row_i == 17:
            ws.cell(row=row_i, column=col).value = f'=((SUM({ws.cell(row=row_i, column=(col)).column_letter}8:{ws.cell(row=row_i, column=(col)).column_letter}10)-SUM({ws.cell(row=row_i, column=(col)).column_letter}5:{ws.cell(row=row_i, column=(col)).column_letter}7))/SUM({ws.cell(row=row_i, column=(col)).column_letter}5:{ws.cell(row=row_i, column=(col)).column_letter}7))'
        if row_i == 18:
            ws.cell(row=row_i, column=col).value = f'=((SUM({ws.cell(row=row_i, column=(col)).column_letter}11:{ws.cell(row=row_i, column=(col)).column_letter}13)-SUM({ws.cell(row=row_i, column=(col)).column_letter}8:{ws.cell(row=row_i, column=(col)).column_letter}10))/SUM({ws.cell(row=row_i, column=(col)).column_letter}8:{ws.cell(row=row_i, column=(col)).column_letter}10))'

for col in range(2,23):
    ws.cell(row=19, column=col).value = f'=AVERAGE({ws.cell(row=19, column=(col)).column_letter}15:{ws.cell(row=19, column=(col)).column_letter}18)'

for col in range(3,23):
    for row_i in range(20,24):
        if row_i == 20:
            ws.cell(row=row_i, column=col).value = f'=(({ws.cell(row=row_i, column=(col)).column_letter}15-{ws.cell(row=row_i, column=(col-1)).column_letter}15)/{ws.cell(row=row_i, column=(col-1)).column_letter}15)'
        if row_i == 21:
            ws.cell(row=row_i, column=col).value = f'=(({ws.cell(row=row_i, column=(col)).column_letter}16-{ws.cell(row=row_i, column=(col-1)).column_letter}16)/{ws.cell(row=row_i, column=(col-1)).column_letter}16)'
        if row_i == 22:
            ws.cell(row=row_i, column=col).value = f'=(({ws.cell(row=row_i, column=(col)).column_letter}17-{ws.cell(row=row_i, column=(col-1)).column_letter}17)/{ws.cell(row=row_i, column=(col-1)).column_letter}17)'
        if row_i == 23:
            ws.cell(row=row_i, column=col).value = f'=(({ws.cell(row=row_i, column=(col)).column_letter}18-{ws.cell(row=row_i, column=(col-1)).column_letter}18)/{ws.cell(row=row_i, column=(col-1)).column_letter}18)'

for col in range(3,23):
    for row_i in range(24,36):
        if row_i == 24:
            ws.cell(row=row_i, column=col).value = f'=AVERAGE({ws.cell(row=row_i, column=(col)).column_letter}2,{ws.cell(row=row_i, column=(col-1)).column_letter}9:{ws.cell(row=row_i, column=(col-1)).column_letter}13)'
        if row_i == 25:
            ws.cell(row=row_i, column=col).value = f'=AVERAGE({ws.cell(row=row_i, column=(col)).column_letter}2:{ws.cell(row=row_i, column=(col)).column_letter}3,{ws.cell(row=row_i, column=(col-1)).column_letter}10:{ws.cell(row=row_i, column=(col-1)).column_letter}13)'
        if row_i == 26:
            ws.cell(row=row_i, column=col).value = f'=AVERAGE({ws.cell(row=row_i, column=(col)).column_letter}2:{ws.cell(row=row_i, column=(col)).column_letter}4,{ws.cell(row=row_i, column=(col-1)).column_letter}11:{ws.cell(row=row_i, column=(col-1)).column_letter}13)'
        if row_i == 27:
            ws.cell(row=row_i, column=col).value = f'=AVERAGE({ws.cell(row=row_i, column=(col-1)).column_letter}12:{ws.cell(row=row_i, column=(col-1)).column_letter}13,{ws.cell(row=row_i, column=(col)).column_letter}2:{ws.cell(row=row_i, column=(col)).column_letter}5)'
        if row_i == 28:
            
            ws.cell(row=row_i, column=col).value = f'=AVERAGE({ws.cell(row=row_i, column=(col)).column_letter}2:{ws.cell(row=row_i, column=(col)).column_letter}6,{ws.cell(row=row_i, column=(col-1)).column_letter}13)'
        if row_i == 29:
            
            ws.cell(row=row_i, column=col).value = f'=AVERAGE({ws.cell(row=row_i, column=(col)).column_letter}2:{ws.cell(row=row_i, column=(col)).column_letter}7)'
        if row_i == 30:
            
            ws.cell(row=row_i, column=col).value = f'=AVERAGE({ws.cell(row=row_i, column=(col)).column_letter}3:{ws.cell(row=row_i, column=(col)).column_letter}8)'
        if row_i == 31:
            
            ws.cell(row=row_i, column=col).value = f'=AVERAGE({ws.cell(row=row_i, column=(col)).column_letter}4:{ws.cell(row=row_i, column=(col)).column_letter}9)'
        if row_i == 32:
            
            ws.cell(row=row_i, column=col).value = f'=AVERAGE({ws.cell(row=row_i, column=(col)).column_letter}5:{ws.cell(row=row_i, column=(col)).column_letter}10)'
        if row_i == 33:
            
            
            ws.cell(row=row_i, column=col).value = f'=AVERAGE({ws.cell(row=row_i, column=(col)).column_letter}6:{ws.cell(row=row_i, column=(col)).column_letter}11)'
        if row_i == 34:
            
            ws.cell(row=row_i, column=col).value = f'=AVERAGE({ws.cell(row=row_i, column=(col)).column_letter}7:{ws.cell(row=row_i, column=(col)).column_letter}12)'
        if row_i == 35:
            
            ws.cell(row=row_i, column=col).value = f'=AVERAGE({ws.cell(row=row_i, column=(col)).column_letter}8:{ws.cell(row=row_i, column=(col)).column_letter}13)'
        

wb.save('remfam2010_2023_datos.xlsx')
wb.close()




Fin de la semi automatizacion del Jupiter Notebook